# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Toronto, Ontario, Canada**, and the stations the data comes from are shown on the map below.

In [6]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
from tabulate import tabulate

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

# leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [45]:
# Load data
binsize = 400
hashid = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89'
df = pd.read_csv(f'data/C2A2_data/BinnedCsvs_d{binsize}/{hashid}.csv')

# Format date to datetime
df['Date'] = pd.to_datetime(df['Date'], yearfirst = True, format='%Y-%m-%d').dt.normalize()

# Note that Data_Value are in tenth of a degree
## Multiple TMIN/TMAX because of the multiple ID per date
# If so, get the max(TMAX) and min(TMIN) for each date

# Extract 2005-2014 time period
df_minmax = (df.loc[(df['Date'].dt.year >= 2005) &
                    (df['Date'].dt.year < 2015) &
                    ((df['Date'].dt.month != 1)&(df['Date'].dt.day != 29))] # Drop any leap year
             .pivot_table(index='Date',columns='Element', values='Data_Value', aggfunc=(min, max)))
df_minmax = df_minmax.loc[:,[('max','TMAX'), ('min','TMIN')]]
df_minmax.columns = df_minmax.columns.droplevel(1)
df_minmax /= 10 # Convert to degrees

# Extract 2015 data
df_2015 = (df.loc[df['Date'].dt.year == 2015]
           .pivot_table(index='Date', columns='Element', values='Data_Value', aggfunc=(max,min)))
df_2015 = df_2015.loc[:,[('max','TMAX'), ('min','TMIN')]]
df_2015.columns = df_2015.columns.droplevel(1)
df_2015 /= 10 # Convert to degrees


df_minmax.index = df_minmax.index.date # remove time portion of datetime format
df_2015.index = df_2015.index.date # remove time portion of datetime format

# print (tabulate(df_2015.head(10), headers = 'keys', tablefmt = 'orgtbl'))
# print('#### #### ####')
# print (tabulate(df_minmax.head(10), headers = 'keys', tablefmt = 'orgtbl'))

|            |   max |   min |
|------------+-------+-------|
| 2015-01-01 |   1.1 | -13.3 |
| 2015-01-02 |   3.9 | -12.2 |
| 2015-01-03 |   3.9 |  -6.7 |
| 2015-01-04 |   4.4 |  -8.8 |
| 2015-01-05 |   2.8 | -15.5 |
| 2015-01-06 |   3.3 | -18.2 |
| 2015-01-07 |  -6.7 | -18.2 |
| 2015-01-08 |  -8.2 | -21.1 |
| 2015-01-09 |  -6.6 | -20.6 |
| 2015-01-10 |  -7.1 | -20.6 |
#### #### ####
|            |   max |   min |
|------------+-------+-------|
| 2005-02-01 |   1.7 | -21.1 |
| 2005-02-02 |   1.7 | -16.7 |
| 2005-02-03 |   6.1 | -16.7 |
| 2005-02-04 |   6.7 | -15.6 |
| 2005-02-05 |   8.9 | -12.8 |
| 2005-02-06 |   7.8 |  -8.9 |
| 2005-02-07 |   6.7 |  -5.6 |
| 2005-02-08 |   6.7 |  -3.3 |
| 2005-02-09 |   3.9 |  -7.8 |
| 2005-02-10 |   0.6 | -11.1 |
